# >Título atractivo

## <span style="color:rgb(204, 153, 0)"> Descripción del juego Clash of Clans</span>
También conocido como CoC, Clash of Clans es un videojuego de estrategia multijugador en el que los jugadores asumen el papel de jefes de clanes y construyen y mejoran sus aldeas. Al mismo tiempo, el usuario puede crear o unirse a un clan, donde podrá interactuar con otros jugadores.

El juego se desarrolla en un entorno multijugador en línea, lo que permite a los jugadores interactuar con otros jugadores de todo el mundo a través de chat y sistema de clanes.

Fue desarrollado por Supercell y lanzado para dispositivos iOS el 2 de agosto del 2012 y el 8 de octubre para dispositivos Android.

Antes de iniciar con el WebScraping, necesitamos crear una cuenta en la web API de Clash of Clans (https://developer.clashofclans.com/#/register) para obtener nuestra clave token.

<img src="https://i.pinimg.com/originals/11/ae/b7/11aeb7ddb3f1d518648a1e36b32b2069.png" alt="Clash of Clans" width="500">

## <span style="color: #597aa2"> Conexión con la url (str) </span>


In [14]:
#Importamos las librerías que vamos a necesitar en nuestro trabajo
import urllib.request
from urllib.request import urlopen
import urllib.parse
import pandas as pd
import requests
import json
import polars as pl
import re

#Abrimos el archivo "key.txt" y leemos el token generado guardado en un archivo texto
with open("keyJimena.txt") as f:
    my_key2 = f.read().rstrip("\n")

#URL base de la API de Clash of Clans
base_url = "https://api.clashofclans.com/v1"

#Parámetro para obtener todos los clanes que tengan 20 o más miembros
endpoint = "/clans?minMembers=20"

#Creamos una solicitud HTTP con la clave de autenticación en el encabezado
request = urllib.request.Request(
    base_url + endpoint,
    None,
    {"Authorization": "Bearer %s" % my_key2}
)

# Hacemos la solicitud y obtenemos la respuesta
uclient = urlopen(request)
responsehtml = uclient.read()
uclient.close()

# Decodificamos la respuesta en formato JSON
datos = responsehtml.decode()

# Tenemos los datos obtenidos
#print(datos)

### <span style="color: #34253B"> Pasar a diccionario (dict) </span>


In [4]:
#Convertimos a diccionario con json ya que esta Api está en este formato
datos_dict = json.loads(datos)
print(type(datos_dict))

<class 'dict'>


### <span style="color: #A6BD4C"> Primer filtrado (df) </span>

In [9]:
#Vamos a filtrar tales aspectos y agruparlos de acuerdo a la cantidad de puntos de los clanes, 
#con el fin de compararlo con Clash Royale

puntos_clanes = []
nombres_clanes = []
tipos = []
location = []
niveles_clanes = []
miembros = []
tag = []

for item in datos_dict["items"]:
    puntos_clanes.append(item['clanPoints'])
    nombres_clanes.append(item['name'])
    tipos.append(item['type'])
    ubicacion = item['location']['name'] if 'location' in item and 'name' in item['location'] else ''
    location.append(ubicacion)
    niveles_clanes.append(item['clanLevel'])
    miembros.append(item['members'])
    tag.append(item['tag'])

#Creamos el diccionario
dff = {
    'ClanPoints CoC': puntos_clanes,
    'Nombre CoC': nombres_clanes,
    'Tipo CoC': tipos,
    'Location CoC': location,
    'Nivel de clan CoC': niveles_clanes,
    'Miembros CoC': miembros,
    'Tag CoC': tag}

### <span style="color: #F34230"> Pasar a dataframe (df) </span>

In [10]:
#Creamos el dataframe con Pandas
DFF = pd.DataFrame(dff)
#print(DFF)

DFF1 = DFF.sort_values(by='ClanPoints CoC', ascending=False).head(10).reset_index(drop=True)
#print(DFF)

primero = DFF1.loc[0, 'Tag CoC']
print(primero)

DFF1

#Observamos el clan con la mayoría de puntos:

#P9PVY999


,ClanPoints CoC,Nombre CoC,Tipo CoC,Location CoC,Nivel de clan CoC,Miembros CoC,Tag CoC
0,37485,LEGEND(●´∀｀●),inviteOnly,Japan,20,33,#P9PVY999
1,37304,Potguys2,closed,Philippines,21,42,#2P92CYLQ
2,36025,KERALA WARHAWKS,inviteOnly,International,13,49,#2YCJURJ98
3,35210,ျမန္မာ့ေသြး,inviteOnly,Myanmar (Burma),20,42,#9LUCQLLR
4,34547,벤젼스,inviteOnly,South Korea,15,33,#PJU99QGG
5,33889,PALAWAN,open,International,12,50,#2LJJ2GV0G
6,32964,MYANMAR EMPEROR,inviteOnly,Myanmar (Burma),17,45,#YUV89JPP
7,32740,IncEssAnt 11/24,inviteOnly,Philippines,21,37,#L28QJRUC
8,32455,Sarim kh,open,Cambodia,14,49,#2Q80JQGGJ
9,31967,SRIMULAT,inviteOnly,International,10,44,#2LP222QYG


## <span style="color: #597aa2"> Segunda conexión con la url (str) </span>

In [15]:
# Codificar el valor de "primero"
tags = urllib.parse.quote(primero)

# Leer la clave de autenticación desde el archivo "keyJimena.txt"
with open("keyJimena.txt") as f:
    my_key2 = f.read().rstrip("\n")

# URL base de la API de Clash of Clans
base_url = "https://api.clashofclans.com/v1"

# Construir el endpoint de la solicitud con el valor codificado
endpoint = "/clans/%s/members?minMembers=20" % tags

# Construir la solicitud con la clave de autenticación
request = urllib.request.Request(
    base_url + endpoint,
    None,
    {
        "Authorization": "Bearer %s" % my_key2
    }
)

# Realizar la solicitud y obtener la respuesta
uclient = urllib.request.urlopen(request)
responsehtml = uclient.read()
uclient.close()

# Decodificar los datos de la respuesta
datos1 = responsehtml.decode()
#print(datos1)

{"items":[{"tag":"#RQJCVLJ0","name":"G A O G A O","role":"coLeader","expLevel":236,"league":{"id":29000022,"name":"Legend League","iconUrls":{"small":"https://api-assets.clashofclans.com/leagues/72/R2zmhyqQ0_lKcDR5EyghXCxgyC9mm_mVMIjAbmGoZtw.png","tiny":"https://api-assets.clashofclans.com/leagues/36/R2zmhyqQ0_lKcDR5EyghXCxgyC9mm_mVMIjAbmGoZtw.png","medium":"https://api-assets.clashofclans.com/leagues/288/R2zmhyqQ0_lKcDR5EyghXCxgyC9mm_mVMIjAbmGoZtw.png"}},"trophies":5371,"builderBaseTrophies":4690,"versusTrophies":4690,"clanRank":1,"previousClanRank":1,"donations":10140,"donationsReceived":7049,"builderBaseLeague":{"id":44000033,"name":"Platinum League II"}},{"tag":"#LUVVJCL8","name":"My Dimension","role":"coLeader","expLevel":252,"league":{"id":29000022,"name":"Legend League","iconUrls":{"small":"https://api-assets.clashofclans.com/leagues/72/R2zmhyqQ0_lKcDR5EyghXCxgyC9mm_mVMIjAbmGoZtw.png","tiny":"https://api-assets.clashofclans.com/leagues/36/R2zmhyqQ0_lKcDR5EyghXCxgyC9mm_mVMIjAbmGo

### <span style="color: #34253B"> Pasar a diccionario (dict) </span>

In [16]:
datos_d = json.loads(datos1)
print(type(datos_d))
datos_d["items"][0]

<class 'dict'>


{'tag': '#RQJCVLJ0',
 'name': 'G A O G A O',
 'role': 'coLeader',
 'expLevel': 236,
 'league': {'id': 29000022,
  'name': 'Legend League',
  'iconUrls': {'small': 'https://api-assets.clashofclans.com/leagues/72/R2zmhyqQ0_lKcDR5EyghXCxgyC9mm_mVMIjAbmGoZtw.png',
   'tiny': 'https://api-assets.clashofclans.com/leagues/36/R2zmhyqQ0_lKcDR5EyghXCxgyC9mm_mVMIjAbmGoZtw.png',
   'medium': 'https://api-assets.clashofclans.com/leagues/288/R2zmhyqQ0_lKcDR5EyghXCxgyC9mm_mVMIjAbmGoZtw.png'}},
 'trophies': 5371,
 'builderBaseTrophies': 4690,
 'versusTrophies': 4690,
 'clanRank': 1,
 'previousClanRank': 1,
 'donations': 10140,
 'donationsReceived': 7049,
 'builderBaseLeague': {'id': 44000033, 'name': 'Platinum League II'}}

## <span style="color:rgb(75, 150, 171)"> Descripción del juego Clash Royale</span>


Tambien conocido como CR es un juego de estrategia y combate en tiempo real ambientada en el mismo universo de Clash of Clans, tambien desarrollada por Supercell. Combina elementos de juegos de cartas coleccionables, defensa de torres y estrategia multijugador en línea.
Antes de iniciar con el WebScraping necesitamos crear una cuenta en la web API https://developer.clashroyale.com/#/ para obtener nuestra clave token.


<img src="https://assets-prd.ignimgs.com/2022/05/27/clashroyale-1653673820137.jpg" alt="Clash of Clans" width="500">

### WebScraping mediante API's [ Clash Royale ]

## <span style="color: #709f1d"> Conexión con la url (str) </span>


In [2]:
import urllib.request
from urllib.request import urlopen
import pandas as pd
import requests
import json
import polars as pl
with open("my_keyArthur.txt") as f:
    my_key =f.read().rstrip("\n")
    
    base_url = "https://api.clashroyale.com/v1"
    
    #endpoint = "/cards"
    endpoint = "/clans?minScore=16000"
    
    request = urllib.request.Request(
        base_url+endpoint,
        None,
        {
            "Authorization": "Bearer %s" % my_key
        }
    )
    uclient=urlopen(request)
    responsehtml=uclient.read()
    #type(responsehtml)
    #uclient.close()
    #data = responsehtml.decode()

    #data
    #response = urllib.request.urlopen(request).read().decode("utf-8")
    
    data = responsehtml#.decode("utf-8")
    uclient.close()
    data
    #data = json.loads(response)
    #for item in data["items"]:
        #print("%s [%d]" % (item["name"], item["maxLevel"]))
        #d ="Clan: %s\nMembers: %d\nScore: %s\nTag: %s\n\n" % (item["name"], item["members"], item["clanScore"], item["tag"])
        #print(d)
   #print(data)

## <span style="color: #26a783"> Pasar a diccionario (dict) </span>

In [3]:
info= json.loads(data)
print(type(info))

<class 'dict'>


## <span style="color: #A6BD4C"> Filtrado (df) </span>

In [11]:
clanPoints = []
location=[]
member=[]
clan=[]
Type=[]
#print(info["items"][0])
for i in range(len(info["items"])):
    clanPoints.append(info["items"][i]['clanScore'])
    location.append(info["items"][i]['location']['name'])
    member.append(info["items"][i]["members"])
    clan.append(info ["items"][i]["name"])
    Type.append(info["items"][i]["type"])
dff={"Clan name (CR)":clan,'ClanPoints (CR)':clanPoints,"Type (CR)":Type,'Location (CR)':location,'Members (CR)':member}
#print(clanPoints)
#print(location)
DFF=pd.DataFrame(dff)

#TOP 10 CLANES PARA LA FECHA 18/06/2023
DFF1 = DFF.sort_values(by='ClanPoints (CR)', ascending=False).head(10).reset_index(drop=True)

DFF1

,Clan name (CR),ClanPoints (CR),Type (CR),Location (CR),Members (CR)
0,Clan #LL8QPC8P,66464,open,United States,50
1,Clan #P9CL8R0G,64456,open,United States,50
2,kutingin,53851,closed,International,47
3,Return,52342,inviteOnly,Spain,50
4,喧嘩稼業,51003,closed,Japan,29
5,Hearthstone BR,49898,closed,Brazil,33
6,아낌없이주는연수,49358,open,International,32
7,ClashWithClass™,46828,closed,Spain,26
8,Acrifaal,45107,closed,Ukraine,21
9,om,43548,closed,International,24


## <span style="color: #ab1ba5"> Pasar a dataframe (df) </span>

In [9]:
DFF=pd.DataFrame(dff)

#TOP 10 CLANES PARA LA FECHA 18/06/2023
DFF1 = DFF.sort_values(by='ClanPoints (CR)', ascending=False).head(10).reset_index(drop=True)

DFF1

,Clan name (CR),ClanPoints (CR),Type (CR),Location (CR),Members (CR)
0,Clan #LL8QPC8P,66464,open,United States,50
1,Clan #P9CL8R0G,64456,open,United States,50
2,kutingin,53851,closed,International,47
3,Return,52342,inviteOnly,Spain,50
4,喧嘩稼業,51003,closed,Japan,29
5,Hearthstone BR,49898,closed,Brazil,33
6,아낌없이주는연수,49358,open,International,32
7,ClashWithClass™,46828,closed,Spain,26
8,Acrifaal,45107,closed,Ukraine,21
9,om,43548,closed,International,24


### >Merge a los DataFrame

### >Pasar a csv

### Errores
-


### >Conclusión